# Exemplos de queries SQL integrados ao Python e Pandas
Usaremos dados de exemplo do site [MySQL Tutorial](http://www.mysqltutorial.org/mysql-sample-database.aspx).

Para recriar o banco de dados em SQLite, sem Python, basta executar o arquivo com os comandos SQL na linha de comando do sistema operacional:

```shell
sqlite3 database.db < sample-database-dump.sql
```

O arquivo `database.db` será criado e tabelas e dados conforme especificados em `sql-sample-database.sql` serão populados.

Se estivéssemos lidando com um SGBD (sistema gerenciador de banco de dados) mais robusto, como MariaDB/MySQL, Oracle ou DB2, o comando acima seria diferente e deverá conter:

* o hostname ou IP do servidor de banco de dados
* usuário e senha de acesso
* nome do banco de dados em que vamos operar

Mas SQLite é bem mais simples e didático e opera num arquivo local.

## Visualizando o banco de dados, suas tabelas e dados

![database example](http://www.mysqltutorial.org/wp-content/uploads/2009/12/MySQL-Sample-Database-Schema.png)

Cada SGBD tem seu próprio ferramental para navegar nos dados. MariaDB/MySQL tem o popular [PHP MyAdmin](https://www.phpmyadmin.net), Oracle e DB2 tem suas próprias ferramentas proprietárias, SQLite tem [DB Browser for SQLite](https://sqlitebrowser.org) e uma ferramenta online chamada [SQLite Online](https://sqliteonline.com).

Para navegar nos dados do DB SQLite que acabamos de criar, use uma das 3 opções:

* Use o comando `sqlite3`.
* Instale o **DB Browser for SQLite** e abra o arquivo.
* Envie o arquivo para o site **SQLiteOnline.com**.

## SQLite

Vamos primeiro usar a estrutura e dados do arquivo `sql-sample-database.sql` para criar um banco de dados no arquivo `database.db`.

In [1]:
import pandas as pd
import sqlite3

db = sqlite3.connect('database.db')

In [2]:
script = 'sample-database-dump.sql'

db.cursor().executescript(open(script).read())

In [3]:
#CRIAR A CONEXÃO COM NOSSA BASSE DE DADOS
conn = sqlite3.connect('database.db')
curs = conn.cursor()


#CRIAR UMA TABELA
curs.execute("""CREATE TABLE IF NOT EXISTS countries (

key INT PRIMARY KEY,

name text UNIQUE,

founding_year INT,

capital text

);""")

conn.commit()

#MOSTRAR UMA TABELA CRIADA
query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,key,name,founding_year,capital


In [4]:
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print (name[0])

my_table
customers
employees
offices
orderdetails
orders
payments
productlines
products
countries


In [5]:
#INSERIR VALORES NA TABELA CRIADA
curs.execute("""INSERT INTO countries  

(key, name, founding_year, capital)

VALUES 

(1, 'BRASIL', 1500, 'SALVADOR')
;""")

conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,key,name,founding_year,capital
0,1,BRASIL,1500,SALVADOR


In [6]:
#INSERIR MAIS VALORES
curs.execute("""INSERT INTO countries  

(key, name, founding_year, capital)

VALUES 

(2, 'MÉXICO', 1519, 'CIDADE DO MÉXICO'),
(3, 'ARGENTINA', 1516, 'BUENOS AIRES')
;""")

conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,key,name,founding_year,capital
0,1,BRASIL,1500,SALVADOR
1,2,MÉXICO,1519,CIDADE DO MÉXICO
2,3,ARGENTINA,1516,BUENOS AIRES


In [7]:
#FAZER UPDATE DE VALORES

curs.execute("""UPDATE countries  

SET capital = 'BRASILIA'

WHERE

name = 'BRASIL' AND
founding_year = 1500

;""")
conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,key,name,founding_year,capital
0,1,BRASIL,1500,BRASILIA
1,2,MÉXICO,1519,CIDADE DO MÉXICO
2,3,ARGENTINA,1516,BUENOS AIRES


In [8]:
#DELETAR UM VALOR
curs.execute("""UPDATE countries  

SET capital = NULL

WHERE

name = 'ARGENTINA'

;""")
conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,key,name,founding_year,capital
0,1,BRASIL,1500,BRASILIA
1,2,MÉXICO,1519,CIDADE DO MÉXICO
2,3,ARGENTINA,1516,None


In [9]:
#DELETAR UMA LINHA
curs.execute("""DELETE FROM countries  

WHERE

name = 'ARGENTINA'

;""")
conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,key,name,founding_year,capital
0,1,BRASIL,1500,BRASILIA
1,2,MÉXICO,1519,CIDADE DO MÉXICO


In [10]:
#DELETAR TABELA
curs.execute("""
DROP TABLE countries;""")

conn.commit()

In [11]:
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print (name[0])

my_table
customers
employees
offices
orderdetails
orders
payments
productlines
products


## Vamos focar em ler dados de um banco relacional utilizando SQL

A operação mais comum em um banco de dados é a leitura de dados, e para isso precisamos escrever um requerimento, que é mais conhecido como QUERY.

Uma query poder ser quebrada em partes:
1. SELECT = irá iniciar a seleção de dados que faremos a leitura
2. FROM = irá selecionar as tabelas de interesse
3. WHERE = vamos declarar as condições para ler os dados
4. GROUP BY = podemos agrupar algum dado considerando uma coluna 
5. Pós processamento = podemos ordenar (ORDER BY) ou estabelecer limites (LIMIT)



## Vamos construir aos poucos nossa query

In [12]:
query = "SELECT * FROM customers;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370.0,118200
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504.0,81700


In [13]:
## vamos ler apenas colunas específicas?

query = "SELECT customerName, phone FROM customers;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,customerName,phone
0,Atelier graphique,40.32.2555
1,Signal Gift Stores,7025551838
2,"Australian Collectors, Co.",03 9520 4555
3,La Rochelle Gifts,40.67.8555
4,Baane Mini Imports,07-98 9555


In [14]:
## vamos utilizar pela primeira vez uma condição para fazer a leitura
query = """SELECT * 
          
          FROM customers 
          
          WHERE country ='USA' ;"""

query2 = pd.read_sql_query(query,db)

query2.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166,71800
1,124,Mini Gifts Distributors Ltd.,Nelson,Susan,4155551450,5677 Strong St.,None,San Rafael,CA,97562,USA,1165,210500
2,129,Mini Wheels Co.,Murphy,Julie,6505555787,5557 North Pendale Street,None,San Francisco,CA,94217,USA,1165,64600
3,131,Land of Toys Inc.,Lee,Kwai,2125557818,897 Long Airport Avenue,None,NYC,NY,10022,USA,1323,114900
4,151,Muscle Machine Inc,Young,Jeff,2125557413,4092 Furth Circle,Suite 400,NYC,NY,10022,USA,1286,138500


In [15]:
## Agora uma condição composta

query = """SELECT * 
          
          FROM customers 
          
          WHERE country ='USA' OR country ='France';"""

query2 = pd.read_sql_query(query,db)

query2.shape

(48, 13)

In [16]:
## quem sabe uma maneira diferente de escrever OR

query = """SELECT * 
          
          FROM customers 
          
          WHERE country NOT IN ('USA', 'France');"""

query2 = pd.read_sql_query(query,db)

query2.shape

(74, 13)

In [17]:
## Uma terceira seleção composta

query = """SELECT * 
          
          FROM customers 
          
          WHERE country ='USA' AND city ='NYC';"""

query2 = pd.read_sql_query(query,db)

query2.shape

(5, 13)

In [18]:
## se tivermos mais de uma tabela e quisermos ler apenas algumas colunas de cada tabela?

query = """SELECT
    o.priceEach,
    p.productCode,
    p.productName
    
    FROM orderdetails AS o, products AS p
    
    WHERE
    o.productCode=p.productCode;"""

queryResult = pd.read_sql_query(query,db)

queryResult.shape

(2996, 3)

In [19]:
# hora de ordenar as linhas pelos valores de uma coluna
# ordem decrescente devemos utilizar o comando DESC após a coluna de referência
query = """SELECT
    o.priceEach,
    p.productCode,
    p.productName
    
    FROM 
    orderdetails AS o, products AS p
    
    WHERE
    o.productCode=p.productCode
    
    ORDER BY
    o.priceEach;"""

queryResult = pd.read_sql_query(query,db)

queryResult.shape

(2996, 3)

In [20]:
# Group by
# Se for usar o Order by junto com o group by, lembrar de colocar o order by após o group by
query = """SELECT
    p.productVendor AS Vendedor,
    SUM(o.priceEach*o.quantityOrdered) AS 'VendaTotal'
    
    FROM 
    orderdetails AS o, products AS p
    
    WHERE
    o.productCode=p.productCode
    
    GROUP BY
    p.productVendor
    
    ORDER BY
    VendaTotal;"""

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,Vendedor,VendaTotal
0,Studio M Art Models,474426.49
1,Red Start Diecast,655487.61
2,Highway 66 Mini Classics,664508.39
3,Carousel DieCast Legends,667190.00
4,Min Lin Diecast,680657.99


In [21]:
# COUNT
query = """SELECT
    p.productVendor AS Vendedor,
    COUNT(p.productVendor) AS 'TotalRegistro'
    
    FROM 
    products AS p
    
    GROUP BY
    p.productVendor
    
    ORDER BY
    TotalRegistro;"""

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,Vendedor,TotalRegistro
0,Red Start Diecast,7
1,Autoart Studio Design,8
2,Min Lin Diecast,8
3,Second Gear Diecast,8
4,Studio M Art Models,8


## Lista completa de agregações 

<img src="list_aggregation.png">

In [22]:
# HAVING
query = """SELECT
    p.productVendor AS Vendedor,
    SUM(o.priceEach*o.quantityOrdered) AS 'VendaTotal'
    
    FROM 
    orderdetails AS o, products AS p
    
    WHERE
    o.productCode=p.productCode
    
    GROUP BY
    p.productVendor
    
    HAVING
    o.quantityOrdered < 30
    
    ORDER BY
    VendaTotal
    ;"""

queryResult = pd.read_sql_query(query,db)

queryResult

,Vendedor,VendaTotal
0,Carousel DieCast Legends,667190.00
1,Autoart Studio Design,736928.03
2,Exoto Designs,793392.31
3,Gearbox Collectibles,828013.76
4,Unimax Art Galleries,884167.33


In [23]:
# limitar a quantidade de linhas que serão lidas
query = """SELECT
    p.productVendor AS Vendedor,
    SUM(o.priceEach*o.quantityOrdered) AS 'VendaTotal'
    
    FROM 
    orderdetails AS o, products AS p
    
    WHERE
    o.productCode=p.productCode
    
    GROUP BY
    p.productVendor
    
    ORDER BY
    VendaTotal DESC
    
    LIMIT
    3;"""

queryResult = pd.read_sql_query(query,db)

queryResult

,Vendedor,VendaTotal
0,Classic Metal Creations,934554.42
1,Unimax Art Galleries,884167.33
2,Gearbox Collectibles,828013.76


## SQLite com Pandas
Agora começamos a fazer queries:

## Lista funcionários por chefe

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
   employees e1,
   employees e2

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
   e1.reportsTo = e2.employeeNumber

MOSTRE AS COLUNAS
   e2.firstName||' '||e2.lastName AS boss,
   e2.jobTitle                    AS department,
   e1.employeeNumber              AS ID,
   e1.firstName||' '||e1.lastName AS employee

ORDENE POR
   boss
```

## Inspeciona pedidos de compra

Dados sobre pedidos aparecem nas tabelas `orders` e `orderdetails` e o que relaciona elas é a coluna `orderNumber`. Vamos inspecionar um único pedido.

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
   orderdetails

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
   orderNumber=10103

MOSTRE AS COLUNAS
    orderLineNumber,
    productCode,
    priceEach,
    quantityOrdered,
    priceEach*quantityOrdered as priceTotal
    
ORDENE POR
   orderLineNumber
```

Vamos melhorar a leitura dessa tabela adicionando a descrição do produto. Faremos um **JOIN** com a tabela `products`.

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
   orderdetails AS o,
   products AS p

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    o.productCode=p.productCode AND
    orderNumber=10103

MOSTRE AS COLUNAS
    o.orderLineNumber,
    o.priceEach,
    o.quantityOrdered,
    o.priceEach*o.quantityOrdered as itemTotal,
    o.productCode,
    p.productName
    
ORDENE POR
   orderLineNumber
```

## Cálculo de totais e subtotais


Performance de venda por categoria de produtos

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails o,
    products p

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    o.productCode=p.productCode

MOSTRE AS COLUNAS
    p.productLine,
    count(p.productLine) AS nItems,
    sum(o.priceEach*o.quantityOrdered)/count(p.productLine) AS averagePerItem,
    sum(o.priceEach*o.quantityOrdered) AS lineTotal

AGRUPE POR
    p.productLine

ORDENE INVERSAMENTE POR
   lineTotal
```

Qual é o produto que mais vende ?

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails o,
    products p

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    o.productCode=p.productCode

MOSTRE AS COLUNAS
    p.productName,
    count(o.quantityOrdered) AS nItems,
#    sum(o.priceEach*o.quantityOrdered) AS lineTotal

AGRUPE POR
    p.productCode

ORDENE INVERSAMENTE POR
   lineTotal
```

In [24]:
query = 

SyntaxError: invalid syntax (<ipython-input-24-aa2fdcdd3a4e>, line 1)

Qual é o cliente que mais compra ?

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails as od,
    orders as o,
    customers as c

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    od.orderNumber = o.orderNumber AND
    c.customerNumber = o.customerNumber

MOSTRE AS COLUNAS
    c.customerNumber,
    c.customerName,
    sum(od.priceEach*od.quantityOrdered) AS customerTotal
    sum(od.quantityOrdered) AS nItems,

AGRUPE POR
    c.customerNumber

ORDENE INVERSAMENTE POR
   customerTotal
```

In [ ]:
query = 

Mostre vendas e valores por mês...

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails as od,
    orders as o,

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    od.orderNumber = o.orderNumber

MOSTRE AS COLUNAS
    substr(o.orderDate,1,7) as month,
    sum(od.priceEach*od.quantityOrdered) AS monthTotal
    sum(od.quantityOrdered) AS nItems,

AGRUPE POR
    month

ORDENE INVERSAMENTE POR
   month
```